In [3]:
import nbimporter
from model import LSTMModel
import torch
import torch.nn as nn
import numpy as np

# Load the saved model from the file named 'model.pth'
model_state_dict = torch.load('/pscratch/sd/h/hsko/jupyter/ML/LSTM/model_gpu/batch_run/cpuDataLoader/model_opt_adam_momentum_05_cri_rmsel1l2_eph_99900_batch_1_scheduledlr_0.01_seg_441_hidden_4_layer_1_dropout_0_logscaleFalse.pth')

# Create a new instance of your model
model = LSTMModel(1,4,10,1,0)

# Load the saved parameters into the model
# model.load_state_dict(model_state_dict)


In [4]:
model.eval()

LSTMModel(
  (lstm): LSTM(1, 4, num_layers=10, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [9]:
from dataset_cpuDataLoader import readAudio
logscale = False
device='cpu'
readaudio = readAudio('/pscratch/sd/h/hsko/jupyter/jupyter/ML/FCN/sound/audio_spectrogram/dataset/DSD100/Mixtures/Dev', '/pscratch/sd/h/hsko/jupyter/jupyter/ML/FCN/sound/audio_spectrogram/dataset/DSD100/Sources/Dev', logscale, device)
NUM_SAMPLE = 1
audios = readaudio.__getitem__(NUM_SAMPLE, 441000)

READING AUDIOS not in LOGSCALE


In [10]:
from torch.utils.data import DataLoader

validation_loader = DataLoader(audios, batch_size=1, shuffle=False, persistent_workers=True, num_workers=8, pin_memory=True)
criterion = nn.MSELoss()

In [11]:
from scipy.io import wavfile

In [12]:
val_loss = []
with torch.no_grad():
    for i, data in enumerate(validation_loader): 
        mixes_uncut = data[0].to(device)
        sources_uncut = list(map(lambda x: x.to(device), data[1]))
        mixes = mixes_uncut
        sources = []
        # sources = sources_uncut[:, start:end, :]
        for tensors in sources_uncut:
            tensor_seg = tensors
            sources.append(tensor_seg)
        mixes_lstm = mixes.unsqueeze(-1).float() # make dimension from (batch size, seq len) to (batch size, seq len, num of input)
        max_mix, _ = torch.max(torch.abs(mixes_lstm), dim=1, keepdims=True)
        mixes_lstm_norm = torch.div(mixes_lstm, max_mix)
        mixes_lstm_norm = torch.nan_to_num(mixes_lstm_norm, nan=0.0)# switch nan to zero

        # sources_lstm = torch.stack([sources[0], sources[1], sources[2], sources[3]], dim=-1).float()
        # sources_lstm = torch.stack([sources[0], sources[1]], dim=-1).float()
        sources_lstm = torch.stack([sources[0]], dim=-1).float()

        # max_src, _ = torch.max(torch.abs(sources_lstm), dim=1, keepdims=True)
        sources_lstm_norm = torch.div(sources_lstm, max_mix)
        sources_lstm_norm = torch.nan_to_num(sources_lstm_norm, nan=0.0)# switch nan to zero

        output = model(mixes_lstm_norm) 
        loss = criterion(output, sources_lstm_norm)
        val_loss.append(loss.item())
        print('Avg val loss=%f'%np.mean(val_loss))
        bass_wav = 'bass.wav'
        # wavfile.write(bass_wav, 44100, output[0,:,0].numpy())
        drums_wav = 'drums.wav'
        # wavfile.write(drums_wav, 44100, output[0,:,1].numpy())
        wavfile.write(drums_wav, 44100, output[0,:,0].numpy())
        

Avg val loss=0.036989


/global/common/software/nersc/pm-2022q4/sw/pytorch/1.13.1/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 441000, 1])) that is different to the input size (torch.Size([1, 441000, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
